# PSA

Some of the randomwall data suffers from the fact inserting momentum is dependent on the staggered point split sources. i.e instead of obtaining e^(-ip.x) as expected you obtain e^(-ip.(x+delta))

In [1]:
import stagcorr
import stagcorr.correlators.main as corr
import importlib as imp
import numpy as np
imp.reload(corr)

<module 'stagcorr.correlators.main' from '/mnt/c/Users/Shaun/Dropbox/Physics/projects/stagcorr/stagcorr/correlators/main.py'>

In [2]:
from re import search
import sys

MILCFILE = 'MILCDATA/MILCRUNS'

def momRead(momLine):

    momKey = momLine.split(":")[1]

    return ",".join(momKey.rstrip("\n").replace(" ", "").replace("[", "").replace("]", "").rstrip(",").split(","))


def STRead(STLine):

    STKey = STLine.split(":")[1]

    return STKey.rstrip("\n").replace(" ", "").replace(",", "")

readq = False
readqExt = False
readaqExt = False

qExtMom = None
aqExtMom = None

qSTExt = None
aqSTExt = None

aqExtSrcT = None

qSourceMom = None
aqSourceMom = None


qST='G5-G5'
NT=1
k=0
take_data = False 
corr_function = []

dataDict = {}

with open(MILCFILE, 'r') as fp:

    for line in fp.readlines():

        data = line.split()
        
        if 'lattice_size' in line:
            NT = int(line.split(",")[-1])
            continue
        
        if 'antiquark_source_origin' in line:
            
            srcT = line[-4]
        
        if 'antiquark_source_mom' in line:
            aqSourceMom = momRead(line)
            continue
        
        if "antiquark_source_label" in line:
            aqSourceType = data[1]
            continue
            
        ####################################

        if 'antiquark_sink_ops' in line:
            readaqExt = True
            continue
            
        if readaqExt and 'spin_taste_extend' in line:
            aqSTExt = STRead(line)
            continue
                
        if readaqExt and 'mom' in line:
            aqExtMom = momRead(line)
            continue
        
        if readaqExt and 't0' in line:
            aqExtSrcT = data[1]
            
            readaqExt = False
            continue
            
            
        ##############################
            
        if 'quark_source_mom' in line:

            qSourceMom = momRead(line)
            continue
        
        ####################################
        
        if 'quark_source_ops' in line:
            readq = True
            continue
        
        if readq and 'spin_taste' in line and 'operation' not in line:
            qST = STRead(line)
            readq = False
            continue
            
                
        ####################################
            
        if 'quark_sink_ops' in line:
            #print(True)
            readqExt = True
            continue
                
        if readqExt and 'mom' in line:
            qExtMom = momRead(line)
            continue
        
        if readqExt and 'spin_taste_extend' in line:
            qSTExt = STRead(line)
            readqExt = False
            continue
        
        
        
            
        ####################################
        
        
        
        if 'momentum:' == data[0]:
            
            momkey = data[1]
            
        if "spin_taste_sink" in line:
            STSink = data[1]
            continue

        if "correlator_key:" in line:
                
            k=0                           
            take_data = True
            
            if aqSourceMom == None:
                if qExtMom == None:
                    aqSourceMom = momkey
                else:
                    aqSourceMom = "0,0,0"
    
                
            if qSourceMom == None:
                if qExtMom != None:
                    pion2Mom = list(map(int,qExtMom.split(",")))
                    pion1Mom = ",".join(map(str, [-1*momval for momval in pion2Mom]))
                    qSourceMom = pion1Mom
                else:
                    qSourceMom = "0,0,0"
            
            if (aqSourceMom, qSourceMom,  qST, STSink, qExtMom, qSTExt, aqExtMom, aqSTExt, momkey, srcT) not in dataDict.keys():
                dataDict[(aqSourceMom, qSourceMom,  qST, STSink, qExtMom, qSTExt, aqExtMom, aqSTExt, momkey, srcT)] = {}
            if aqSourceType not in dataDict[(aqSourceMom, qSourceMom,  qST, STSink, qExtMom, qSTExt, aqExtMom, aqSTExt, momkey, srcT)].keys():
                dataDict[(aqSourceMom, qSourceMom,  qST, STSink, qExtMom, qSTExt, aqExtMom, aqSTExt, momkey, srcT)][aqSourceType] = {}
            continue
            
        if bool(search(r'\d', line)) == True and take_data:
            data = line.split()

            corr_function.append( float(data[1]) + 1j* float(data[2]))

            k+=1


        if k==NT and take_data:
            dataDict[(aqSourceMom, qSourceMom,  qST, STSink, qExtMom, qSTExt, aqExtMom, aqSTExt, momkey, srcT)][aqSourceType][aqExtSrcT] = corr_function

            take_data = False
            corr_function = []

            aqSourceMom, qSourceMom,  qST, STSink, qExtMom, qSTExt, aqExtMom, aqSTExt, momkey, srcT, aqExtSrcT = None, None, None, None, None, None, None, None, None, None, None
            
            qST='G5-G5'
                
                

In [3]:
signDict = {}

vol = (6,6,6,6)

#Sym p1(0)p2(1) -> p3(t)p4(t+1)
#Asym p1(0)p2(1) -> p3(t+1)p4(t)

for (aqSourceMom, qSourceMom,  qST, STSink, qExtMom, qSTExt, aqExtMom, aqSTExt, momkey, srcT), corrFuncDict in dataDict.items():
    
    
    if qExtMom == None:
        
        incSpin, incT = qST.split("-")
        if aqSourceMom == '0,0,0' and qSourceMom == '0,0,0':
            incMom = [0,0,0]
            outMom = [0,0,0]
        
        elif aqSourceMom == '0,0,0' and qSourceMom != '0,0,0':
            incMom = list(map(int,qSourceMom.split(",")))
            outMom = [-1*momval for momval in incMom]
        
        elif aqSourceMom != '0,0,0' and qSourceMom == '0,0,0':
            incMom = list(map(int,aqSourceMom.split(",")))
            outMom = [-1*momval for momval in incMom]
        
        
        two_point_pion = corr.generate_npt(spinTasteMassNaikMomSymShift1=[incSpin, incT, 1, 0, incMom,0],
                                           spinTasteMassNaikMomSymShift2=[incSpin, incT, 1, 0, outMom,0],
                                           volume=vol)
        
        res= np.real(two_point_pion[:,0])/np.prod(vol[1:])
        
        ratio = res / np.real(corrFuncDict['pt_src'][None])
        if np.sign(ratio[0]) == 1:
            signMatch = "SAME SIGN"
            if qST not in signDict.keys():
                signDict[qST] = {}
            signDict[qST]['2pt'] = 1
        else:
            signMatch = "DIFF SIGN"
            if qST not in signDict.keys():
                signDict[qST] = {}
            signDict[qST]['2pt'] = -1
        
        print('Two Point: ' + qST + " p" + str("".join(map(str,incMom))) + " srcTime: " + srcT +  " | " + signMatch)
        #print("")
        print("My res: " + str(res[:3]))
        print("MILC ptSrc: " + str(np.real(corrFuncDict['pt_src'][None])))
        print("MILC RW: " + str(np.real(corrFuncDict['RW'][None])))
        print("My Res / MILC : " + str(ratio[:3]))
        print("")
        
        
        
    elif aqExtMom == None:
        
        incSpin, incT = qST.split("-")
        sinkSpin, sinkT = STSink.split("-")
        
        if aqSourceMom == '0,0,0' and qSourceMom != '0,0,0':
            incMom = list(map(int,qSourceMom.split(",")))
            outMom = [-1*momval for momval in incMom]
        
        elif aqSourceMom != '0,0,0' and qSourceMom == '0,0,0':
            incMom = list(map(int,aqSourceMom.split(",")))
            outMom = [-1*momval for momval in incMom]
        
        
        
        three_point_pion = corr.generate_npt(spinTasteMassNaikMomSymShift1=[incSpin, incT, 0.1, 0, incMom,0],
                                              spinTasteMassNaikMomSymShift2=[incSpin, incT, 0.1, 0, outMom,0],
                                              spinTasteMassNaikMomSymShift3=[sinkSpin, sinkT, 0.1, 0, [0,0,0],0],
                                              volume=vol)

        res = np.imag(three_point_pion[:,1,0]/np.prod(vol[1:]))
        
        ratio = res / np.imag(corrFuncDict['pt_src'][None])
        if np.sign(ratio[0]) == 1:
            signMatch = "SAME SIGN"
            if qST not in signDict.keys():
                signDict[qST] = {}
            signDict[qST]['3pt'] = 1
        else:
            signMatch = "DIFF SIGN"
            if qST not in signDict.keys():
                signDict[qST] = {}
            signDict[qST]['3pt'] = -1
        
        print('Three Point: ' + qST + " p" + str("".join(map(str,incMom))) + " srcTime: " + srcT + " | " + signMatch)
        #print("")
        print("My res: " + str(res[:3]))
        print("MILC ptSrc: " + str(np.imag(corrFuncDict['pt_src'][None][:3])))
        print("MILC RW: " + str(np.imag(corrFuncDict['RW'][None][:3])))
        print("Ratio to MILC pt: " + str(ratio[:3]))
        print("")
        
    else:

        incSpin, incT = qST.split("-")
        sinkSpin, sinkT = STSink.split("-")
        
        
        if '-' not in aqExtMom:
            #diagram=O
            #t (t3) = aqExtSrcT
            #t4 = tie up time
            dia = 'O'
            p1IncMom = list(map(int,qSourceMom.split(",")))
            p2IncMom = list(map(int,qExtMom.split(","))) 
            p3OutMom = [-1*pi for pi in list(map(int,aqExtMom.split(",")))] #MILC antiquark flips mom sign
            p4OutMom = list(map(int,momkey.split(",")))
            
            # O goes p1<->p2<->p4<->p3 as p1 <-> p3 / p2 <-> p4 make normal 2points
            four_point_pion = np.real(corr.generate_npt(spinTasteMassNaikMomSymShift1=[incSpin, incT, 0.1, 0, p1IncMom,0],
                                              spinTasteMassNaikMomSymShift2=[incSpin, incT, 0.1, 0, p2IncMom,0],
                                              spinTasteMassNaikMomSymShift3=[sinkSpin, sinkT, 0.1, 0,p4OutMom, 0],
                                              spinTasteMassNaikMomSymShift4=[sinkSpin, sinkT, 0.1, 0, p3OutMom,0],
                                              volume=vol))/np.prod(vol[1:])
            
            resSym2 = four_point_pion[1,2,1,0]
            resSym3 = four_point_pion[2,3,1,0]
            resSym4 = four_point_pion[3,4,1,0]
            resSym5 = four_point_pion[4,5,1,0]
            

            resSymMILCpt2 = np.real(corrFuncDict['pt_src']['1'][2])
            resSymMILCpt3 = np.real(corrFuncDict['pt_src']['2'][3])
            resSymMILCpt4 = np.real(corrFuncDict['pt_src']['3'][4])
            resSymMILCpt5 = np.real(corrFuncDict['pt_src']['4'][5])
            
            resSymMILCrw2 = np.real(corrFuncDict['RW']['1'][2])
            resSymMILCrw3 = np.real(corrFuncDict['RW']['2'][3])
            resSymMILCrw4 = np.real(corrFuncDict['RW']['3'][4])
            resSymMILCrw5 = np.real(corrFuncDict['RW']['4'][5])
            
            ratioSym2 = resSym2 / resSymMILCpt2
            ratioSym3 = resSym3 / resSymMILCpt3
            ratioSym4 = resSym4 / resSymMILCpt4
            ratioSym5 = resSym5 / resSymMILCpt5

            
            if np.sign(ratioSym2) == 1:
                signMatchS = "SAME SIGN"
                if qST not in signDict.keys():
                    signDict[qST] = {}
                signDict[qST]['4ptO'+'->'+STSink] = 1
                
            else:
                signMatchS = "DIFF SIGN"
                if qST not in signDict.keys():
                    signDict[qST] = {}
                signDict[qST]['4ptO'+'->'+STSink] = -1
                
    
            
            print('Four Point Sym O: ' + qST + " p" + str("".join(map(str,p1IncMom))) + " -> " + STSink + " p" + str("".join(map(str,p4OutMom))) + " srcTime: " + srcT + " | " + signMatchS)
            #print("")
            print("My res: " + str([resSym2, resSym3, resSym4, resSym5]))
            print("MILC ptSrc: " + str([resSymMILCpt2, resSymMILCpt3, resSymMILCpt4, resSymMILCpt5]))
            print("MILC RW: " + str([resSymMILCrw2, resSymMILCrw3, resSymMILCrw4, resSymMILCrw5]))
            print("Ratio to MILC: " + str([ratioSym2, ratioSym3, ratioSym4, ratioSym5]))
            print("")
        
        else:
            #diagram=8
            #(t4) = aqExtSrcT
            #t=t3 = tie up time
            dia = '8'
            p1IncMom = list(map(int,qSourceMom.split(",")))
            p2IncMom = list(map(int,qExtMom.split(","))) 
            p3OutMom = p2IncMom #MILC antiquark flips mom sign
            p4OutMom = list(map(int,momkey.split(",")))
            
            

            four_point_pion = np.real(corr.generate_npt(spinTasteMassNaikMomSymShift1=[incSpin, incT, 0.1, 0, p1IncMom,0],
                                              spinTasteMassNaikMomSymShift2=[incSpin, incT, 0.1, 0, p2IncMom,0],
                                              spinTasteMassNaikMomSymShift3=[sinkSpin, sinkT, 0.1, 0,p3OutMom, 0],
                                              spinTasteMassNaikMomSymShift4=[sinkSpin, sinkT, 0.1, 0, p4OutMom,0],
                                              volume=vol))/np.prod(vol[1:])
            
            resSym2 = four_point_pion[2,1,1,0]
            resSym3 = four_point_pion[3,2,1,0]
            resSym4 = four_point_pion[4,3,1,0]
            resSym5 = four_point_pion[5,4,1,0]
            

            resSymMILCpt2 = np.real(corrFuncDict['pt_src']['2'][1])
            resSymMILCpt3 = np.real(corrFuncDict['pt_src']['3'][2])
            resSymMILCpt4 = np.real(corrFuncDict['pt_src']['4'][3])
            resSymMILCpt5 = np.real(corrFuncDict['pt_src']['5'][4])
            
            resSymMILCrw2 = np.real(corrFuncDict['RW']['2'][1])
            resSymMILCrw3 = np.real(corrFuncDict['RW']['3'][2])
            resSymMILCrw4 = np.real(corrFuncDict['RW']['4'][3])
            resSymMILCrw5 = np.real(corrFuncDict['RW']['5'][4])
            
            ratioSym2 = resSym2 / resSymMILCpt2
            ratioSym3 = resSym3 / resSymMILCpt3
            ratioSym4 = resSym4 / resSymMILCpt4
            ratioSym5 = resSym5 / resSymMILCpt5
            
            
            if np.sign(ratioSym2) == 1:
                signMatchS = "SAME SIGN"
                if qST not in signDict.keys():
                    signDict[qST] = {}
                signDict[qST]['4pt8'+'->'+STSink] = 1
                
            else:
                signMatchS = "DIFF SIGN"
                if qST not in signDict.keys():
                    signDict[qST] = {}
                signDict[qST]['4pt8'+'->'+STSink] = -1
                
       
            print('Four Point Sym 8: ' + qST + " p" + str("".join(map(str,p1IncMom))) + " -> " + STSink + " p" + str("".join(map(str,p4OutMom))) + " srcTime: " + srcT + " | " + signMatchS)
            #print("")
            print("My res: " + str([resSym2, resSym3, resSym4, resSym5]))
            print("MILC ptSrc: " + str([resSymMILCpt2, resSymMILCpt3, resSymMILCpt4, resSymMILCpt5]))
            print("MILC RW: " + str([resSymMILCrw2, resSymMILCrw3, resSymMILCrw4, resSymMILCrw5]))
            print("Ratio to MILC rw: " + str([ratioSym2, ratioSym3, ratioSym4, ratioSym5]))
            print("")
        
        
       

/home/shaun/my-venv/lib/python3.12/site-packages/numpy/_core/getlimits.py:551: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


Two Point: G5-G5 p000 srcTime: 0 | SAME SIGN
My res: [0.1885994  0.01904623 0.00214502]
MILC ptSrc: [0.1885994  0.01904623 0.00214502 0.0007103  0.00214502 0.01904623]
MILC RW: [0.1888318  0.0192132  0.00217957 0.00074814 0.00217957 0.0192132 ]
My Res / MILC : [1.00000002 0.99999983 1.00000015]

Two Point: GX-G1 p000 srcTime: 0 | SAME SIGN
My res: [0.08420306 0.01162087 0.00124205]
MILC ptSrc: [0.08420306 0.01162087 0.00124205 0.0005763  0.00124205 0.01162087]
MILC RW: [0.08312736 0.0117315  0.00127539 0.00061557 0.00127539 0.0117315 ]
My Res / MILC : [0.99999995 1.00000012 1.00000003]

Two Point: GX-G1 p000 srcTime: 1 | SAME SIGN
My res: [0.08420306 0.01162087 0.00124205]
MILC ptSrc: [0.08420306 0.01162087 0.00124205 0.0005763  0.00124205 0.01162087]
MILC RW: [0.08247006 0.01142534 0.00120911 0.00055543 0.00120911 0.01142534]
My Res / MILC : [0.99999995 1.00000012 1.00000003]

Three Point: G5-G5 p001 srcTime: 0 | SAME SIGN
My res: [-0.00114853 -0.00114853  0.00019313]
MILC ptSrc: [-0.

Four Point Sym O: G5-G5Z p001 -> G5-G5Z p001 srcTime: 0 | SAME SIGN
My res: [np.float64(0.00034013515605923155), np.float64(-3.329074965870892e-05), np.float64(0.00017375980082313898), np.float64(-3.329074965870892e-05)]
MILC ptSrc: [4.954856e-05, 1.788688e-06, 7.315741e-05, 1.788688e-06]
MILC RW: [-0.0003402847, 3.094217e-05, -0.0001901856, 3.094217e-05]
Ratio to MILC: [np.float64(6.8646829707913115), np.float64(-18.611825907429868), np.float64(2.3751497055887976), np.float64(-18.611825907429868)]

Four Point Sym O: G5T-G5T p001 -> G5T-G5T p001 srcTime: 0 | SAME SIGN
My res: [np.float64(0.0014501441248964706), np.float64(-0.0001459279998067253), np.float64(0.000703053736530932), np.float64(-0.0001459279998067253)]
MILC ptSrc: [0.001450144, -0.000145928, 0.0007030537, -0.000145928]
MILC RW: [0.001456745, -0.0001374943, 0.0007695707, -0.0001374943]
Ratio to MILC: [np.float64(1.0000000861269436), np.float64(0.9999999986755475), np.float64(1.0000000519603724), np.float64(0.999999998675547